# CoLab Download Utilities

Useful scripts for using CoLab as a remote dowwnloader

In [ ]:
##############################################
## Code for setting up SSH server on the VM ##
##############################################

## You will be given almost all permissions in the VM including installing
## programs, starting services, etc. Some functionalities are forbidden e.g.
## iptables. However, it is not a practice of honor to take advantage of a free
## scientific computing platform to download unrelevent in the first place, so
## it is not recommanded to actively exploit it in the way of a VPS. After all
## we don't want to push too far to Google's limit which could be responded
## with a strict ban.

import random, string, urllib.request, json, getpass

# Generate root password
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))

# Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip

# Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null

# Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

# Add X11 forwarding support
! echo "X11Forwarding yes" >> /etc/ssh/sshd_config
! echo "X11UseLocalhost no" >> /etc/ssh/sshd_config

# Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

# Ask token
print("Copy authtoken from https://dashboard.ngrok.com/auth")
authtoken = getpass.getpass()

# Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')

# Get public address and print connect command
with urllib.request.urlopen('http://localhost:4040/api/tunnels') as response:
  data = json.loads(response.read().decode())
  (host, port) = data['tunnels'][0]['public_url'][6:].split(':')
  print(f'SSH command: ssh -p{port} root@{host}')

# Print root password
print(f'Root password: {password}')

In [ ]:
####################################
## Mount Google Drive to CoLab VM ##
####################################

import os
from google.colab import drive  
      # Load the Drive helper and mount
mountpoint = '/content/drive'
if not os.path.isdir(os.path.join(mountpoint, 'My Drive')):
  drive.mount(mountpoint, force_remount=True)

In [ ]:
#####################################
## Install aria2 as the downloader ##
#####################################

## There are notebooks shared all over the internect that uses libtorrent in
## python, but why bother when you can set up a full function downloader? In
## this way, you can plug in the aria2 commands just like what you did in your
## local machine.

! sudo apt install aria2
! cd /content && aria2c https://raw.githubusercontent.com/ngosang/trackerslist/master/trackers_best_ip.txt -o tracker --allow-overwrite="true" -q

In [ ]:
#####################################
## Enter the link to be downloaded ##
#####################################

with open("/content/download_list", "w+") as f:
  while True:
      magnet_link = input("Enter Magnet Link Or Type Exit: ")
      if magnet_link.lower() == "exit":
          break
      print(magnet_link, end="\n\n", file=f)

! cd /content/drive/MyDrive/Downloads/Torrent && aria2c --bt-tracker=$(sed ':a;N;$!ba;s/\n\n/,/g' /content/tracker) --bt-enable-lpd=true --disable-ipv6 --seed-time=0 --file-allocation=none --console-log-level=warn -i /content/download_list    

In [ ]:
###########################
## Peek downloaded video ##
###########################

## The syncing process between CoLab and Google Drive can be time-consuming and
## you would want to peek into the download file to see if it contains the right
## content. 

from IPython.display import HTML
from base64 import b64encode
import os

# Input video path
save_path = input("Enter the path to the file to peek (you can find it in the left penel): ")

# Compressed video path
compressed_path = "/content/result_compressed.mp4"

# It is necessary to extract only a short clip and encod it into mp4 before it
# can be 
os.system(f"ffmpeg -i {save_path} -ss 00:00:30 -to 00:01:00 -strict -2 -vcodec libx264 -acodec copy {compressed_path}")
print("Compression finished")

# Show video
print(r"If no video shown. Please download /content/result_compressed.mp4 " + 
      "from the left panel and verify it locally.")
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
#######################################
## Manually sync VM and Google Drive ##
#######################################

## This is to make sure all modification done in CoLab have been updated to the
## Google drive. You need to rerun the mounting script before next download.

drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')
drive.mount(mountpoint, force_remount=True)

In [ ]:
####################
## Setting up RPC ##
####################

# For more complex download tasks like transfering data drom Onedrive to Google
# drive, It is recommended to use the RPC deamon and ariang web UI which make
# configuration a lot easier.

os.system(f"aria2c --enable-rpc --rpc-listen-all -d /content/drive/MyDrive/Downloads/ --disable-ipv6 --rpc-secret={password} --max-concurrent-downloads=10 --max-connection-per-server=10 --min-split-size=10M --split=5  --bt-tracker=$(sed ':a;N;$!ba;s/\n\n/,/g' /content/tracker) --bt-enable-lpd=true --rpc-allow-origin-all --file-allocation=none --seed-time=300 -D")
print("Please connect to the VM through the following command:")
print(f'ssh "{host}" -p "{port}" -L 8080:localhost:80 -L 6800:localhost:6800 -l "root"')
print(f'Root password: {password}')